In [64]:
# Steps
# Connect to the Vicuna model to generate the questions
import os
import openai
from dotenv import load_dotenv
load_dotenv()
from langchain.llms.base import LLM, BaseLanguageModel
from typing import Optional, List, Mapping, Any
import requests
import json
from collections import deque
TOGETHER_AI_BEARER_TOKEN= os.getenv("TOGETHER_AI_BEARER_TOKEN") # Bitte hier den API Key von TogetherAI eintragen
openai.api_key = os.getenv("OPENAI_API_KEY") # Bitte hier den API Key von OpenAI eintragen


class TogetherLLM(LLM):
    bearerToken:str 
    server_url:str = "https://api.together.xyz/inference"
    pload={
        "model": "togethercomputer/llama-2-70b-chat",
        "max_tokens": 500,
        "prompt": "Hi, how are you?",
        "request_type": "language-model-inference",
        "temperature": 0.0,
        "top_p": 0.7,
        "top_k": 50,
        "repetition_penalty": 1,
        "stop": [
            "\n### Benutzer:"
        ],
        "safety_model": "",
        "repetitive_penalty": 1,
        "update_at": "2023-09-11T13:47:21.562Z"
    }
    
    model:str = "togethercomputer/llama-2-70b-chat"
    @property
    def _llm_type(self) -> str:
        
        return "vicuna"
    
    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        self.pload["prompt"] = prompt
        if stop is None or len(stop) == 0:
            stop = ["\n### Benutzer:"]
        #self.pload["stop"] = stop[0]
        self.pload["model"] = self.model
        return self.ask_chatbot_without_stream(self.pload)

      
    def ask_chatbot_without_stream(self, pload):
        headers={"Authorization": f"Bearer {self.bearerToken}"}
        response = requests.post(
            self.server_url,
            json=pload,
            headers=headers
        )
        if response.status_code == 200:
            return(response.json()['output']['choices'][0]['text'].strip())
        else:
            return("Error in generation "+ response.status_code)

In [65]:
llm = TogetherLLM(bearerToken=TOGETHER_AI_BEARER_TOKEN)

In [66]:
def generate_questions(prompt:str, paragraph:str):
    prompt_full = "### Benutzer:" + prompt + "\nAbsatz:" +paragraph + "\n### Assistant: Hier sind die Fragen:\n 1."
    return llm(prompt_full,stop=["\n### Benutzer:"])

In [67]:
def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]


In [73]:
prompt = "Du bist der Assistent, du machst genau was der Benutzer sagt. Bitte generiere maximal 10 Fragen, die der Benutzer mit Hilfe des Absatzes beantworten kann und deren Antworten jeweils aus einem Satz bestehen. Achte darauf dass sie immer Fragen sind. Die Fragen sollten immer kurz und knapp sein. Beginne mit Definitionen."

In [86]:
paragraph ="Johann Wolfgang von Goethe wurde am 28. August 1749 im Goetheschen Familienhaus (dem heutigen Goethe-Haus) am Frankfurter Großen Hirschgraben geboren und tags darauf evangelisch getauft. Sein Rufname war Wolfgang. Sein aus Thüringen stammender Großvater Friedrich Georg Göthe (1657–1730) hatte sich 1687 als Schneidermeister in Frankfurt niedergelassen und die Schreibweise des Familiennamens geändert.[2] Später bot sich ihm die Gelegenheit, in ein florierendes Gasthaus- und Herbergsgeschäft einzuheiraten. Als Gastwirt und Weinhändler war er zu einem ansehnlichen Vermögen gekommen, das er in Gestalt von Immobilien, Hypothekarkrediten und mehreren Säcken voller Geld seinen beiden Söhnen aus erster Ehe und dem jüngsten Sohn Johann Caspar Goethe (1710–1782), Johann Wolfgang Goethes Vater, hinterließ.[3] Goethes Vater hatte zwar an der Leipziger Universität den Doktorgrad der Jurisprudenz erworben, übte aber keinen juristischen Beruf aus. Mit dem Ehrentitel „Kaiserlicher Rat“ stieg er in die Frankfurter Oberschicht auf. Als Rentier lebte er von den Erträgen seines ererbten Vermögens, das später auch dem Sohn Leben und Studium ohne finanzielle Zwänge ermöglichen sollte.[4] Er war vielseitig interessiert und gebildet, jedoch auch streng und pedantisch, was wiederholt zu Konflikten in der Familie führte."

In [87]:
questions = generate_questions(prompt,paragraph)

In [88]:
questions = " 1. " + questions 

In [89]:
questions

' 1. Wann wurde Johann Wolfgang von Goethe geboren?\n 2. Wo wurde Johann Wolfgang von Goethe geboren?\n 3. Welchen Rufnamen hatte Johann Wolfgang von Goethe?\n 4. Woher stammte Goethes Großvater?\n 5. Was war der Beruf von Goethes Vater?\n 6. Welchen Ehrentitel trug Goethes Vater?\n 7. Wie lebte Goethes Vater?\n 8. Was ermöglichte Goethes Vater dem Sohn?\n 9. Was war Goethes Vater?\n 10. Wie war Goethes Vater?\n### Benutzer:'

In [90]:
questions_list = questions.split("\n")

In [91]:
questions_list

[' 1. Wann wurde Johann Wolfgang von Goethe geboren?',
 ' 2. Wo wurde Johann Wolfgang von Goethe geboren?',
 ' 3. Welchen Rufnamen hatte Johann Wolfgang von Goethe?',
 ' 4. Woher stammte Goethes Großvater?',
 ' 5. Was war der Beruf von Goethes Vater?',
 ' 6. Welchen Ehrentitel trug Goethes Vater?',
 ' 7. Wie lebte Goethes Vater?',
 ' 8. Was ermöglichte Goethes Vater dem Sohn?',
 ' 9. Was war Goethes Vater?',
 ' 10. Wie war Goethes Vater?',
 '### Benutzer:']

In [92]:
questions_list.pop()

'### Benutzer:'

In [93]:
filtered_list = [x for x in questions_list if x != '']

In [94]:
filtered_list

[' 1. Wann wurde Johann Wolfgang von Goethe geboren?',
 ' 2. Wo wurde Johann Wolfgang von Goethe geboren?',
 ' 3. Welchen Rufnamen hatte Johann Wolfgang von Goethe?',
 ' 4. Woher stammte Goethes Großvater?',
 ' 5. Was war der Beruf von Goethes Vater?',
 ' 6. Welchen Ehrentitel trug Goethes Vater?',
 ' 7. Wie lebte Goethes Vater?',
 ' 8. Was ermöglichte Goethes Vater dem Sohn?',
 ' 9. Was war Goethes Vater?',
 ' 10. Wie war Goethes Vater?']

In [95]:
eval_prompt ='Ich gebe dir den Kontext, die Frage und die Antwort. Bewerte die Antwort, indem du die Wahrscheinlichkeiten für die Klassen in einer neuen Zeile angibst, wobei die Wahrscheinlichkeiten zwischen 0 und 1 liegt. Die Klassen sind - Contradiction für widersprüchlich oder falsch, Neutral und Entailment für richtig. Gebe auch den Grund für die Punktzahl an. Gebe mir die Ausgabe in einem json-String bestehend aus {"contradiction":value, "neutral":value, "entailment": value, "reasoning_behind_grading": value}. Gebe eine nachvollziehbare Erklärung für die vergebene Bewertung und hilfreiche Hinweise, wie die Antwort verbessert werden kann. Die Begründung soll eloquent und auf Deutsch sein. Es wird nur die Ausgabe von json-Strings akzeptiert, kein zusätzlicher Text. Die Summe der Wahrscheinlichkeiten aller Klassen sollte 1 sein.'

In [96]:
def evaluate(eval_prompt:str, question:str, paragraph:str, user_answer:str):
    prompt_full = eval_prompt + "\nFrage: " + question + "\nAbsatz: " + paragraph + "\nAntwort von Benutzer: " + user_answer
    bewertung = get_completion(prompt_full)
    return bewertung


In [111]:
frage_nummer = 0 # Bitte Wählen Sie eine Frage aus der Liste aus
mein_antwort ="Johann Wolfgang von Goethe wurde im 1749 geboren." # Bitte geben Sie Ihre Antwort ein



In [112]:
json_ans = evaluate(eval_prompt, questions_list[frage_nummer], paragraph, mein_antwort)

In [114]:
type(json_ans)

str

In [115]:
import json
json_ans = json.loads(json_ans) if type(json_ans) == str else json_ans

In [116]:
from IPython.display import JSON
JSON(json_ans)

<IPython.core.display.JSON object>

In [117]:

# Zusammenfassung
JSON({"Deine_Frage":questions_list[frage_nummer],"Deine_Antwort":mein_antwort,"Bewertung":json_ans})

<IPython.core.display.JSON object>